# Dales Omuse interface basics

## Imports

In [ ]:
import numpy 
from omuse.units import units
from omuse.community.dales.interface import Dales
import matplotlib
import matplotlib.pyplot as plt
import os


## Initialization

In [ ]:
# create a dales object

d=Dales(redirection='none')

# alternative for redirecting Dales output to files
#d=Dales(redirection='file', redirect_stdout_file='dales.out', redirect_stderr_file='dales.err')

# Set parameters
d.parameters_DOMAIN.itot = 32  # number of grid cells in x
d.parameters_DOMAIN.jtot = 32  # number of grid cells in y
d.parameters_DOMAIN.xsize = 6400 | units.m
d.parameters_DOMAIN.ysize = 6400 | units.m

d.parameters_RUN.ladaptive = True
# d.parameters_RUN.nsv = 2  # number of scalar variables - needed for microphysics


# Select advection schemes
d.parameters_DYNAMICS.iadv_mom = 62
d.parameters_DYNAMICS.iadv_thl = 52
d.parameters_DYNAMICS.iadv_qt = 52
d.parameters_DYNAMICS.iadv_tke = 52
d.parameters_DYNAMICS.iadv_sv = [52,52]

# d.parameters_NAMMICROPHYSICS.imicro=6


# Nudging towards given profiles

In [ ]:
# temporary - to get the model in a state where we can set nudging profiles
d.evolve_model(1 | units.s)

# create a profile
kmax = d.parameters_DOMAIN.kmax
QTn = numpy.ones(kmax) * 0.005 | units.mfu
QTn[5:15] *= 1.5
d.nudge.QT = QTn

# note, some ways of setting the target profile do NOT work currently
# d.nudge.QT[5] = 0.005 | units.mfu  # does not work
# maybe one has to assign to the whole grid, not just elements ??


# set time scale of nudging
d.set_nudge_time_QT(600 | units.s)

print(d.nudge.QT)



In [ ]:
# plot profiles - before time step
plt.plot(d.grid.QT[2,2,:].value_in(units.shu))
plt.plot(d.profile_grid.QT[:].value_in(units.shu))
plt.ylim((0.,0.01))

In [ ]:
# time stepping
d.evolve_model(300 | units.s)

In [ ]:
# plot profiles - after time stepping
plt.plot(d.grid.QT[2,2,:].value_in(units.shu))
plt.plot(d.profile_grid.QT[:].value_in(units.shu))
plt.ylim((0.,0.01))

In [ ]:
# get an xz crossection
crossection = d.grid.THL[:,1,:].value_in(units.K) 
plt.imshow(crossection.T, origin='lower')
plt.colorbar()

plt.figure()
crossection = d.grid.QT[:,1,:].value_in(units.shu) 
plt.imshow(crossection.T, origin='lower')
plt.colorbar()